# Running Tune experiments with SigOpt

This example demonstrates the usage of SigOpt with Ray Tune. Please note that usage of `SigOpt` requires an API key.

We also combine the search algorithm based on `SigOptSearch` with `AsyncHyperBandScheduler` scheduler to demonstrate Ray Tune's modularity.

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

Necessary requirements:
- `pip install ray[tune]`
- `pip install sigopt==7.5.0`

In [ ]:
# !pip install ray[tune]
!pip install sigopt==7.5.0

In [ ]:
import time
import os

import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.sigopt import SigOptSearch

if "SIGOPT_KEY" not in os.environ:
    raise ValueError(
        "SigOpt API Key not found. Please set the SIGOPT_KEY "
        "environment variable."
    )

Let's start by defining a simple evaluation function.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment and try to tune two hyperparameters,
namely `width` and `height`, and `activation`.

In [ ]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 1
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)

In [ ]:
ray.init(configure_logging=False)

Next we define the search algorithm built from `SigOptSearch`, constrained  to a maximum of `1` concurrent trials.

In [ ]:
space = [
    {
        "name": "width",
        "type": "int",
        "bounds": {"min": 0, "max": 20},
    },
    {
        "name": "height",
        "type": "int",
        "bounds": {"min": -100, "max": 100},
    },
]

algo = SigOptSearch(
    space,
    name="SigOpt Example Experiment",
    max_concurrent=1,
    metric="mean_loss",
    mode="min",
) 

Furthermore, we define a `scheduler` to go along with our algorithm to showcase the modularity of Ray Tune.

In [ ]:
scheduler = AsyncHyperBandScheduler() 

The number of samples this Tune run is set to `1000`.
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# If 1000 samples take too long, you can reduce this number.
# We override this number here for our smoke tests.
num_samples = 10

Finally, all that's left is to define a search space.

In [ ]:
search_config = {
    "steps": 100,
    "width": tune.randint(0, 10),
    "height": tune.quniform(-10, 10, 1e-2),
    "activation": tune.choice(["relu, tanh"])
}

And run the experiment.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="sigopt_exp",
    num_samples=num_samples,
    config=search_config,
)

Here are the hyperparamters found to minimize the mean loss of the defined objective.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

## Multi-objective optimization with Sigopt

We define another simple objective.

In [ ]:
np.random.seed(0)
vector1 = np.random.normal(0, 0.1, 100)
vector2 = np.random.normal(0, 0.1, 100)

def evaluate(w1, w2):
    total = w1 * vector1 + w2 * vector2
    return total.mean(), total.std()


def multi_objective(config):
    w1 = config["w1"]
    w2 = config["total_weight"] - w1
    
    average, std = evaluate(w1, w2)
    tune.report(average=average, std=std, sharpe=average / std)
    time.sleep(0.1)

We define the space manually for `SigOptSearch`

space = [
    {
        "name": "w1",
        "type": "double",
        "bounds": {"min": 0, "max": 1},
    },
]

algo = SigOptSearch(
    space,
    name="sigopt_multiobj_exp",
    observation_budget=num_samples,
    max_concurrent=1,
    metric=["average", "std", "sharpe"],
    mode=["max", "min", "obs"],
)

analysis = tune.run(
    multi_objective,
    name="sigopt_multiobj_exp",
    search_alg=algo,
    num_samples=num_samples,
    config={"total_weight": 1},
)

print("Best hyperparameters found were: ", analysis.get_best_config("average", "min"))
```

## Incorporating prior beliefs with Sigopt

We start with defining another objective.

```python
np.random.seed(0)
vector1 = np.random.normal(0.0, 0.1, 100)
vector2 = np.random.normal(0.0, 0.1, 100)
vector3 = np.random.normal(0.0, 0.1, 100)

def evaluate(w1, w2, w3):
    total = w1 * vector1 + w2 * vector2 + w3 * vector3
    return total.mean(), total.std()

def multi_objective_two(config):
    w1 = config["w1"]
    w2 = config["w2"]
    total = w1 + w2
    if total > 1:
    w3 = 0
    w1 /= total
    w2 /= total
    else:
    w3 = 1 - total
    
    average, std = evaluate(w1, w2, w3)
    tune.report(average=average, std=std)
```

Now we set up the experiment.

In [ ]:
samples = num_samples

conn = Connection(client_token=os.environ["SIGOPT_KEY"])
experiment = conn.experiments().create(
    name="prior experiment example",
    parameters=[
        {
            "name": "w1",
            "bounds": {"max": 1, "min": 0},
            "prior": {"mean": 1 / 3, "name": "normal", "scale": 0.2},
            "type": "double",
        },
        {
            "name": "w2",
            "bounds": {"max": 1, "min": 0},
            "prior": {"mean": 1 / 3, "name": "normal", "scale": 0.2},
            "type": "double",
        },  
    ],
    metrics=[
        dict(name="std", objective="minimize", strategy="optimize"),
        dict(name="average", strategy="store"),
    ],
    observation_budget=samples,
    parallel_bandwidth=1,
)

algo = SigOptSearch(
    connection=conn,
    experiment_id=experiment.id,
    name="sigopt_prior_multi_exp",
    max_concurrent=1,
    metric=["average", "std"],
    mode=["obs", "min"],
)

analysis = tune.run(
    easy_objective, name="my_exp", search_alg=algo, num_samples=samples, config={}
)

print("Best hyperparameters found were: ", analysis.get_best_config("average", "min"))

In [ ]:
ray.shutdown()